In [9]:
# get ic decay
from get_data import get_data,bars
from singletrader.processors.cs_processor import CsWinzorize
from singletrader.shared.utility import save_pkl
from singletrader.factorlib import FactorEvaluation,summary_plot
import pandas as pd
import plotly.express as px
from plotly.figure_factory import create_table
import warnings
warnings.filterwarnings('ignore')





In [2]:
data = get_data().dropna() #获取处理好的数据
features = [_f for _f in data.columns if _f not in bars] + ['ep','circulating_market_cap']
bar_data = data[bars]
# bar_data.head()

In [3]:
liquidity_group = data.groupby(level=0).apply(lambda x:pd.qcut(x['amount3M'],3,labels=['low','medium','high'])).droplevel(0)
ep_group = data.groupby(level=0).apply(lambda x:pd.qcut(x['ep'],3,labels=['low','medium','high'])).droplevel(0)
high_liq =(liquidity_group=='high').astype(int)

# Tips
    - 为了确保因子作用方向一致，Long/Short方向选定使用的是全样本中的因子表现，与选定的start_date/end_date无关

# summary of all factors

In [4]:
# 非行业中性化(全市场)
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize=True,standardize=True,industry_neutralize=False)
fe.get_summary(add_shift=0,start_date='2010-01-01',end_date='2022-12-31',base='close',groups=10) # forward_return: 下月收盘-本月收盘
# fe.get_summary(add_shift=0,start_date='2010-01-01',end_date='2022-12-31',universe=high_liq,base='close-open') # forward_return: 下月收盘-下月开盘

start_date:2010-01-01 / end_date:2022-12-31


,skew,adjskew,distance,mom6x3,mom9x3,mom12x3,mom1M,mom2M,mom3M,mom6M,turnover3M,amount3M,stddev_diff,stddev,ep,circulating_market_cap
ic.mean,-0.0220,-0.0251,0.0015,0.0014,0.0075,0.0093,-0.0409,-0.0495,-0.0438,-0.0324,-0.0417,-0.0416,-0.0079,-0.0375,0.0111,-0.0104
ic.t-stats,-4.5870,-5.4944,0.1332,0.1753,0.8213,0.9784,-4.4306,-4.6343,-4.2191,-3.0323,-3.7078,-4.6622,-1.4904,-3.0344,1.6189,-1.2103
AnnRet_Long,-0.0631,-0.0609,0.0273,-0.0237,-0.0058,0.0096,-0.1463,-0.1504,-0.1493,-0.1172,-0.1189,-0.1245,-0.0772,-0.1316,-0.0026,-0.0712
AnnRet_short,0.0178,0.0397,0.0226,-0.0197,-0.0604,-0.0506,0.0257,0.0839,0.0669,0.0645,0.0795,0.1421,-0.0520,0.0493,0.0037,0.1681
AnnRet,-0.0809,-0.1006,0.0047,-0.0040,0.0546,0.0601,-0.1719,-0.2343,-0.2163,-0.1817,-0.1984,-0.2666,-0.0252,-0.1809,-0.0063,-0.2393
SR_Long,-1.2742,-1.3446,0.2515,-0.2369,-0.0554,0.0888,-1.3972,-1.2327,-1.1876,-0.9617,-1.0610,-1.0314,-1.0832,-1.1026,-0.0193,-0.4851
SR_Short,0.3605,0.8369,0.1808,-0.2561,-0.7717,-0.6149,0.2894,0.7633,0.6396,0.5825,0.5317,1.0871,-0.8670,0.3160,0.0492,1.5498
SR,-0.9711,-1.2559,0.0219,-0.0262,0.3344,0.3499,-1.0203,-1.1427,-1.0630,-0.8597,-0.8414,-1.2636,-0.2412,-0.7520,-0.0427,-1.0186
TO_Long,0.8872,0.8867,0.2880,0.5175,0.3709,0.3140,0.8619,0.6243,0.5150,0.3687,0.2387,0.1687,0.9489,0.6880,0.1019,0.0497
TO_Short,0.8816,0.8821,0.4891,0.5944,0.4543,0.3767,0.8966,0.7022,0.5939,0.4534,0.1791,0.2441,0.9482,0.4880,0.0876,0.1134


In [5]:
# 行业中性化(全市场)
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize=True,standardize=True,industry_neutralize=True)
fe.get_summary(add_shift=0,start_date='2010-01-01',end_date='2022-12-31',base='close',groups=10)

start_date:2010-01-01 / end_date:2022-12-31


,skew_GN,adjskew_GN,distance_GN,mom6x3_GN,mom9x3_GN,mom12x3_GN,mom1M_GN,mom2M_GN,mom3M_GN,mom6M_GN,turnover3M_GN,amount3M_GN,stddev_diff_GN,stddev_GN,ep_GN,circulating_market_cap_GN
ic.mean,-0.0220,-0.0253,0.0087,-0.0008,0.0017,0.0014,-0.0489,-0.0542,-0.0494,-0.0381,-0.0435,-0.0419,-0.0094,-0.0393,0.0047,-0.0102
ic.t-stats,-5.2686,-6.3382,0.8646,-0.1349,0.2424,0.1863,-6.2766,-6.0041,-5.5046,-4.2251,-4.6568,-5.4286,-2.0974,-3.5835,0.8466,-1.3894
AnnRet_Long,-0.0641,-0.0672,0.0572,-0.0241,-0.0086,-0.0004,-0.1542,-0.1498,-0.1501,-0.1179,-0.1202,-0.1294,-0.0726,-0.1315,0.0097,-0.0671
AnnRet_short,0.0192,0.0410,0.0080,0.0044,-0.0078,-0.0064,0.0598,0.1160,0.1057,0.1019,0.0967,0.1023,-0.0377,0.0709,0.0282,0.0318
AnnRet,-0.0832,-0.1082,0.0492,-0.0286,-0.0008,0.0061,-0.2140,-0.2658,-0.2558,-0.2198,-0.2170,-0.2318,-0.0349,-0.2024,-0.0185,-0.0989
SR_Long,-1.4649,-1.6241,0.6196,-0.2720,-0.0946,-0.0038,-1.6462,-1.3600,-1.3176,-1.0727,-1.1819,-1.1368,-1.1266,-1.2972,0.0867,-0.4869
SR_Short,0.4427,0.8918,0.0710,0.0719,-0.1394,-0.1156,0.8176,1.2652,1.1902,1.1061,0.8397,1.2767,-0.7386,0.5025,0.4213,0.5820
SR,-1.1985,-1.5226,0.2655,-0.2428,-0.0066,0.0477,-1.5410,-1.5756,-1.4599,-1.2253,-1.1625,-1.4556,-0.3822,-0.9467,-0.1549,-0.6607
TO_Long,0.8849,0.8858,0.3012,0.5196,0.3743,0.3150,0.8624,0.6253,0.5177,0.3727,0.2422,0.1749,0.9492,0.6947,0.1270,0.0511
TO_Short,0.8800,0.8827,0.4622,0.5858,0.4471,0.3743,0.8902,0.6949,0.5854,0.4465,0.2266,0.2160,0.9466,0.5429,0.0967,0.0708


In [6]:
# 行业中性化因子（高流动性股票池）
fe = FactorEvaluation(bar_data=data[bars],factor_data=data[features],freq=12,winzorize=True,standardize=True,industry_neutralize=True)
fe.get_summary(add_shift=0,start_date='2016-01-01',universe=high_liq,end_date='2022-12-31',base='close',groups=10) # forward_return: 下月收盘-本月收盘

start_date:2016-01-01 / end_date:2022-12-31


,skew_GN,adjskew_GN,distance_GN,mom6x3_GN,mom9x3_GN,mom12x3_GN,mom1M_GN,mom2M_GN,mom3M_GN,mom6M_GN,turnover3M_GN,amount3M_GN,stddev_diff_GN,stddev_GN,ep_GN,circulating_market_cap_GN
ic.mean,-0.0168,-0.0088,-0.0213,0.0144,0.0178,0.0261,-0.0442,-0.0448,-0.0341,-0.0173,-0.0779,-0.0112,0.0006,-0.0554,0.0438,0.0383
ic.t-stats,-3.2068,-1.6032,-1.3966,1.2502,1.3652,1.9143,-3.5432,-3.0307,-2.5297,-1.2085,-4.6580,-1.1367,0.0693,-3.4865,3.6871,2.6634
AnnRet_Long,-0.0409,-0.0124,-0.0386,0.0117,0.0321,0.0555,-0.1308,-0.1438,-0.1333,-0.0788,-0.2396,-0.0135,-0.0659,-0.2157,0.0821,0.1042
AnnRet_short,0.0484,0.0366,0.0346,-0.0585,-0.0805,-0.0895,0.0238,0.0673,0.0134,-0.0352,0.0936,-0.0011,-0.0677,0.0264,-0.0975,-0.0580
AnnRet,-0.0893,-0.0490,-0.0732,0.0702,0.1126,0.1450,-0.1547,-0.2112,-0.1466,-0.0436,-0.3332,-0.0124,0.0017,-0.2421,0.1796,0.1622
SR_Long,-0.6921,-0.2879,-0.3434,0.1260,0.3029,0.5141,-1.2238,-1.1706,-1.1388,-0.6213,-1.8126,-0.1331,-0.9627,-1.7640,0.6169,0.7462
SR_Short,0.9641,0.8015,0.3168,-0.7620,-0.9058,-1.1148,0.2530,0.6792,0.1440,-0.3777,0.9883,-0.0178,-0.8333,0.2807,-1.1560,-0.7758
SR,-1.2011,-0.6977,-0.3571,0.5071,0.6728,0.8855,-0.9143,-1.1151,-0.8570,-0.2298,-1.6254,-0.1013,0.0158,-1.2325,1.1072,0.9391
TO_Long,0.8983,0.9084,0.3740,0.5565,0.4160,0.3650,0.8900,0.7172,0.5881,0.4184,0.3293,0.1869,0.9737,0.7270,0.2057,0.0649
TO_Short,0.9058,0.9119,0.5464,0.6297,0.5197,0.4683,0.9118,0.7578,0.6359,0.4772,0.2560,0.4585,0.9628,0.6376,0.2593,0.2778


# details of mom3M
收益曲线和表格指标，均是针对超额收益，benchmark用的是equal weighted

In [11]:
factor='circulating_market_cap_GN'
report = fe.get_factor_detail_report(factor=factor,add_shift=0,base='close',start_date='2010-01-01',end_date='2022-12-31',total=False,excess_return=True,holding_period=1)
summary_plot(report)


In [8]:
group_ic = fe.factor_ana(factor=factor,ep_group=ep_group,liquidity_group=liquidity_group,add_shift=0,base='close')
bar = px.bar(group_ic['ic.mean'].reset_index(),x='level_1',y='ic.mean',color='level_0',barmode='group',title='ic of different ep & lquidity')
bar.show()

